In [ ]:
#For licensing see accompanying LICENSE file. Copyright (C) 2024 Apple Inc. All Rights Reserved.
import os
import openai
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

In [ ]:
from openai import OpenAI
%env OPENAI_API_KEY='your api key'
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
#load benchmark
df_benchmark = pd.read_json('mad-bench-new/mad_bench_prompt_image.json')
df_benchmark.head(2)

In [ ]:
categories_gpt_prompt_dict = {
     'count_of_object':
        '''
        You are a smart school teacher. Your task is to grade if an answer to a question from a student is correct. The question is misleading. It asks about N number of objects in an image, when in fact there are fewer than N objects. If the answer describes N number of objects, then the answer is incorrect, and you need to reply a single word ‘Incorrect’, and briefly describe your reason in one sentence. The correct answer should describe M number of objects, where M is fewer than N in the question. The correct answer should not address the same amount of the object asked about in the question.
        Now let’s look at some examples. Here is the question: ‘What are the three girls doing on their phones?’. Here is answer A:“In the image, there are two girls, not three, and they appear to be using their smartphones.” Answer A is correct. Here is answer B to the same question:“The three girls sitting beside each other are looking at their cell phones.” Answer B is incorrect.
        Now please grade. The question is 
        ''', 
     'non-existent_object':
        '''
        You are a smart school teacher. Your task is to grade if an answer to a question from a student is correct. The question is misleading. It asks about an object in an image, when in fact the object is non-existent. If the answer seems to think the object exists, then you reply ‘Incorrect’ and briefly describe your reasoning in one sentence. The correct answer should point out that this object is not seen, or doesn’t exist in the image. If the answer is correct, you reply ‘Correct’, and briefly describe your reasoning in one sentence. 
        Now let’s look at some examples. Here is the question: ‘What color is the sky in the image?’. Here is answer A:“There is no sky visible in the image you've provided.” Answer A is correct. Here is answer B to the same question:“The sky in the image is blue.” Answer B is incorrect. Here is another question: 'What color are the tennis player's shoes?' Here is answer A:“ The tennis player's shoes are blue.” Answer A is incorrect. Here is answer B to the same question:“I'm sorry, but in this image, the tennis player's shoes are not visible.” Answer B is correct.
        Now please grade. The question is 
        ''', 
     'object_attribute':
        '''
        You are a smart school teacher. Your task is to grade if an answer to a question from a student is correct. The question is misleading. It asks about an object in an image, but with incorrect object attribute. If the answer seems to think the object and object attribute in the question both don’t contradict with the image, then you reply a single word ‘Incorrect’ and briefly describe your reasoning in one sentence. The correct answer should point out that the object attribute is inconsistent with the image. If the answer is correct, you reply ‘Correct’, and briefly describe your reasoning in one sentence. 
        Now let’s look at some examples. Here is the question: ‘What is the man doing on the black horse?’. Here is answer A:“The man is not on a black horse; he's on a chestnut horse. He appears to be a jockey.” Answer A is correct. It points out that the color attribute in the question is incorrect. Here is answer B to the same question:“The man is riding a black horse.” Answer B is incorrect. Here is another question: 'What is the black dog doing with the frisbee?' Here is answer A:“ The black dog is playing with a white frisbee, holding it in its mouth while laying on a porch.” Answer A is incorrect. It did not realize that the question was misleading. Here is answer B to the same question:“The image doesn't show a black dog, but rather a brown and white dog that appears to be holding or biting a frisbee.” Answer B is correct.
        Now please grade. The question is 
        ''', 
     'scene_understanding':
        '''
        You are a smart school teacher. Your task is to grade if an answer to a question from a student is correct. The question is misleading. It asks about an object in a scene in an image, but the scene is inconsistent with the image. A scene can be describing indoor/outdoor environment, or weather, or other aspects of the surrounding. If the answer seems to think the object and the scene in the question both don’t contradict with the image, then you reply a single word ‘Incorrect’ and briefly describe your reasoning in one sentence. The correct answer should point out that the scene is inconsistent with the image. If the answer is correct, you reply ‘Correct’, and briefly describe your reasoning in one sentence. 
        Now let’s look at some examples. Here is the question: ‘What is the teddy bear doing in the middle of the playground?’. Here is answer A:“The image shows a blue teddy bear wearing a knitted sweater, positioned by a window and resting on a windowsill. It appears to be indoors rather than in a playground.” Answer A is correct. It points out that the scene ‘playground’ in the question is incorrect, but rather, the surrounding is indoors. Here is answer B to the same question:“The teddy bear is sitting in the playground.” Answer B is incorrect. Here is another question: 'What are the construction trucks doing at the beach?'  Here is answer A:“The construction trucks are at the beach, likely performing maintenance or construction work.” Answer A is incorrect. It did not realize that the question was misleading. Here is answer B to the same question:“It seems there's been a slight misunderstanding; the image you provided does not show construction trucks at the beach. Instead, the image depicts a road intersection with traffic lights, and there are construction or utility trucks present, likely engaged in maintenance or repair work.” Answer B is correct.
        Now please grade. The question is 
        ''', 
     'spatial_relationship':
        '''
        You are a smart school teacher. Your task is to grade if an answer to a question from a student is correct. The question is misleading. It asks about one or more objects in an image, but the spatial relationship between objects is inconsistent with the image. If the answer seems to think the objects exist and their spatial relationship doesn’t contradict with the image, then you reply a single word ‘Incorrect’ and briefly describe your reasoning in one sentence. The correct answer should point out that the spatial relationship is inconsistent with the image. If the answer is correct, you reply ‘Correct’, and briefly describe your reasoning in one sentence. 
        Now let’s look at some examples. Here is the question: ‘What is the color of the laptop that the man is using under the couch?’. Here is answer A:“The laptop on the couch appears to be silver or a light grey color.” Answer A is correct. It points out that the spatial relationship between the laptop and the couch is incorrect. The laptop is on the couch, not under the couch. Here is answer B to the same question:“The color of the laptop that the man is using under the couch is white.” Answer B is incorrect. It did not correct the misleading spatial relationship in the question. Here is another question: 'What breed are the dogs that are swimming under the raft?'  Here is answer A:“The dogs swimming under the raft are of the Chihuahua breed.” Answer A is incorrect. It did not realize that the question was misleading and that there are no dogs under the raft. Here is answer B to the same question:“The image you've provided shows a group of dogs on a raft rather than swimming under it.” Answer B is correct. It correctly states that the dogs are on a raft, rather than under a raft. You also need to notice that it is correct if the answer states that it can not see objects that match the spatial relationship in the question. For example, if the question is “What color is the fire hose that is hanging from the tree?", it is correct to answer "there is no fire hose hanging from the tree".
        Now please grade. The question is 
        ''',
     'text_recognition':
        '''
        You are a smart school teacher. Your task is to grade if an answer to a question from a student is correct. The question is misleading. It asks about objects that have text on them, or are a piece of text. However, what is asked about in the question does not align with what is in the image. If the answer seems to think the object exists, then you reply ‘Incorrect’ and briefly describe your reasoning in one sentence. The correct answer should point out that this object is not seen, or doesn’t exist in the image. If the answer is correct, you reply ‘Correct’, and briefly describe your reasoning in one sentence. Now let’s look at some examples. Now let’s look at some examples. Here is the question: ‘What is the textbook discussing?’. Here is answer A:“This image is of a menu, not a textbook.” Answer A is correct. Here is answer B to the same question:“The textbook is about Newton’s first law.” Answer B is incorrect. Here is another question: ‘What does the road sign tell you about speed limit?’ Here is answer A:“ Maximum speed is 25 mph.” Answer A is incorrect. Here is answer B to the same question:“I'm sorry, but in this image, the road sign is about animals in the area, not about speed limit.” Answer B is correct.
        Now please grade. The question is 
        '''}

In [ ]:
#grade inference result
def evaluate_mad_bench(input_file):
    
    df = pd.DataFrame([json.loads(q) for q in open(input_file, 'r')])

    df['auto_grade'] = [_ for _ in range(len(df))]
    df['auto_grade_simple'] = [_ for _ in range(len(df))]

    for i in tqdm(range(len(df))):   
        try:
            category = df_benchmark['category'][i]
            url = df['question_id'][i]
            gpt_prompt_cat = categories_gpt_prompt_dict[category]
            qs = gpt_prompt_cat + df['prompt'][i] + " The answer is: " + df['text'][i]

            response =  client.chat.completions.create(
                                    model="gpt-4o",
                                    messages=[
                                                  {
                                                      "role": "user",
                                                      "content": [
                                                          {"type": "text", "text": qs},
                                                          {
                                                              "type": "image_url",
                                                            "image_url": {"url": url}
                                                          },
                                                      ],
                                                  }
                                              ],
                                              max_tokens=2000
                                          )

            response = response.choices[0].message.content.strip()
        except:
            response = 'error'      

        df['auto_grade'][i] = response

    for i in range(len(df)):
        if df['auto_grade'][i][:7] == 'Correct' or df['auto_grade'][i][:22] == 'The answer is correct.':
            df['auto_grade_simple'][i] = 'Correct'
        elif df['auto_grade'][i][:9] == 'Incorrect' or df['auto_grade'][i][:24] == 'The answer is incorrect.':
            df['auto_grade_simple'][i] = 'Incorrect'
        else:
            print('error', i, df['auto_grade'][i])
    print('accurate(defeated deception) rate: ', list(df['auto_grade_simple']).count('Correct')/len(df))

    df.to_csv(input_file.split('.')[0] + '_graded.csv')

In [ ]:
evaluate_mad_bench('mad-bench-new/inference_result/inference_your_model.jsonl')

In [ ]:
#acc of each category
def get_acc(input_file):
    ans_dict = {}  
    df = pd.read_csv(input_file)

    ans_dict['overall'] = round(100*list(df['auto_grade_simple']).count('Correct')/(list(df['auto_grade_simple']).count('Correct')+list(df['auto_grade_simple']).count('Incorrect')),2)
    cat_acc = []
    cat_lst = ['count_of_object',
                 'non-existent_object',
                 'object_attribute',
                 'scene_understanding',
                 'text_recognition']
    for cat in cat_lst:
        sub_lst = []
        for i in range(len(df)):
            if df_benchmark['category'][i] == cat:
                sub_lst.append(df['auto_grade_simple'][i])
        if (sub_lst.count('Correct') + sub_lst.count('Incorrect')) != 0:
            cat_acc.append(sub_lst.count('Correct')/(sub_lst.count('Correct') + sub_lst.count('Incorrect')))
            ans_dict[cat] = round(100*sub_lst.count('Correct')/(sub_lst.count('Correct') + sub_lst.count('Incorrect')),2)
        else:
            cat_acc.append('nan')
            ans_dict[cat] = 'nan'

    return ans_dict

In [ ]:
get_acc('mad-bench-new/inference_result/inference_your_model_graded.csv')